In [ ]:
!pip install xgboost scikit-learn joblib pandas numpy fastapi uvicorn pyngrok

In [2]:
!pip install python-multipart

In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks/"

/content/drive/MyDrive/Colab Notebooks


In [4]:
from pyngrok import ngrok

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import pandas as pd
import joblib

file_path = '/content/drive/MyDrive/Colab Notebooks/Laptop_price.csv'
df = pd.read_csv(file_path)
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])


pipeline = Pipeline([
    ('preprocessor', preprocessor),
   ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, '/content/drive/MyDrive/Colab Notebooks/laptop_price_model.pkl')

['/content/drive/MyDrive/Colab Notebooks/laptop_price_model.pkl']

In [7]:
%%writefile app.py
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO

app = FastAPI()

# Загрузка обученной модели
model_path = "/content/drive/MyDrive/Colab Notebooks/laptop_price_model.pkl"
model = joblib.load(model_path)

@app.get('/')
async def root():
    return {'hello': 'world'}

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}

Writing app.py


In [8]:
from google.colab import userdata
import os
os.environ["NGROK"] = userdata.get('NGROK')
os.environ["GITHUB_NAME"] = userdata.get('GITHUB_NAME')
os.environ["GITHUB_EMAIL"] = userdata.get('GITHUB_EMAIL')
os.environ["GITHUB_TOKEN"] = userdata.get('GITHUB_TOKEN')

In [9]:
!ngrok config add-authtoken $NGROK

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)

API доступно по адресу: NgrokTunnel: "https://78a4-34-145-187-167.ngrok-free.app" -> "http://localhost:8000"


In [11]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &

In [12]:
!git config --global user.name $GITHUB_NAME
!git config --global user.email $GITHUB_EMAIL

!git init
!git add .
!git commit -m "Добавлен ML-пайплайн"

!git push https://${GITHUB_TOKEN}@github.com/UserStipa/mtuci-homework-1.git master

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Colab Notebooks/.git/
[master (root-commit) 61ef09c] Добавлен ML-пайплайн
 5 files changed, 1026 insertions(+)
 create mode 100644 Laptop_price.csv
 create mode 100644 app.py
 create mode 100644 fastapi.log
 create mode 100644 laptop_price_model.pkl
 create mode 100644 mtuci-homework.ipynb
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100